In [1]:
import numpy as np
import pandas as pd


In [55]:
df = pd.read_csv('returns.csv', header=None)


In [56]:
# ai
def sample_cov(df):
    cov = np.array(df - df.mean(axis=0))
    total = np.zeros((cov.shape[1], cov.shape[1]))
    for r in range(cov.shape[0]):
        total += cov[[r], :].T @ cov[[r], :]
    return total/cov.shape[0]


sample_covariance = sample_cov(df)
np.savetxt('sample_covariance.csv', sample_covariance, delimiter=',')


In [221]:
def run(S, lam=0.00001, iterations = 100, stopping_thres = 1e-12):
    T, N = df.shape
    J = np.eye(N)
    def assign(i,j, new):
        J[i][i] = (new)[0][0]
        J[i][j] = (new)[0][1]
        J[j][i] = (new)[1][0]
        J[j][j] = (new)[1][1]
    def obj():
        return np.log(np.linalg.det(J))-np.trace(J @ S) - lam * np.sum(np.abs(J - np.diag(np.diag(J))))
        # return np.log(np.linalg.det(J))-np.trace(J @ S) - lam * (sum(sum(abs(J))) - np.trace(abs(J)))
    prev_obj = obj()
    for _ in range(iterations):
        for i in range(N):
            for j in range(i+1,N):
                Jbb = J[[a for a in range(N) if not a in [i,j]], :][:, [a for a in range(N) if not a in [i,j]]]
                Jab = J[[i, j], :][:, [a for a in range(N) if not a in [i,j]]]
                Jba = J[[a for a in range(N) if not a in [i,j]], :][:, [i, j]]
                L = Jab@np.linalg.inv(Jbb)@Jba
                Saa = S[[i, j], :][:, [i, j]]
                Svv = S[j][j]
                Suu = S[i][i]
                ms = []
                try: 
                    m1 = np.linalg.inv(Saa+lam*np.array([[0, 1], [1, 0]]))
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m1) > 0):
                        ms.append(m1)
                try: 
                    m2 = np.linalg.inv(Saa+lam*np.array([[0, -1], [-1, 0]]))
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m2) > 0):
                        ms.append(m2)
                try: 
                    m3 = np.array([[(1+np.sqrt(1+4*Svv*Suu*L[0][1]**2))/(2*Suu), -L[0][1]],
                                [-L[1][0], (1+np.sqrt(1+4*Svv*Suu*L[0][1]**2))/(2*Svv)]])
                except: 
                    pass
                else:
                    if np.all(np.linalg.eigvals(m3) > 0) and np.any(np.isnan(m3)):
                        ms.append(m3)
                curr_obj = obj()
                for m in ms:
                    prev_Jaa = J[[i,j],:][:,[i,j]]
                    assign(i, j, m+L)
                    new_obj = obj()
                    if new_obj>curr_obj:
                        curr_obj = new_obj
                    else:
                        assign(i, j, prev_Jaa)
        # if obj()-prev_obj < stopping_thres:
            # break
        if curr_obj-prev_obj > stopping_thres:
            prev_obj = curr_obj
            print(curr_obj)
        else:
            break
    print(np.count_nonzero(J))
    return J


In [222]:
J_test = run(sample_covariance, 0, 1)

384.9659145662428
2500


In [223]:
new_J1 = run(sample_covariance, 0.0001)

362.3984234065307
363.6768462728107
363.90236730601015
364.0119481847503
364.11304847565094
364.1348322452991
364.1454160628752
364.15240374721833
364.15783590005555
364.16038677416384
364.1617593533045
364.1620887485549
364.16217960129853
364.16220045663187
364.1622072870239
364.16220986609346
364.16221080941705
364.1622111509731
364.16221128490724
364.1622113465553
364.16221137864704
364.1622113957761
364.1622114045183
364.16221140866736
364.16221141048294
364.16221141122264
364.1622114115114
364.1622114116318
364.1622114116847
364.1622114117114
364.1622114117268
364.1622114117337
364.16221141173503
364.16221141173696
774


In [224]:
new_J2 = run(sample_covariance, 0.001)

50


In [225]:
print(np.count_nonzero(np.linalg.inv(new_J)))
print(np.count_nonzero(np.linalg.inv(new_J1)))
print(np.count_nonzero(np.linalg.inv(new_J2)))

2500
2500
50


In [214]:
from sklearn.covariance import graphical_lasso
sample_covariance = df.cov().to_numpy()
S, J = graphical_lasso(sample_covariance, 0.00001, cov_init=None)
print(np.count_nonzero(J)) # 1132
print(np.count_nonzero(S)) # 2500
S1, J1 = graphical_lasso(sample_covariance, 0.0001, cov_init=None)
print(np.count_nonzero(J1)) # 538
print(np.count_nonzero(S1)) # 1942
S2, J2 = graphical_lasso(sample_covariance, 0.001, cov_init=None)
print(np.count_nonzero(J2)) # 50
print(np.count_nonzero(S2)) # 50

1132
2500
538
1942
50
50
